In [2]:
import pandas as pd
import numpy as np

## 2-3 단기 체류 외국인 (월별 합산)

In [4]:
data2_3 = pd.read_csv("files\\data\\2-3_short_term_frgn.csv", encoding='cp949')
data2_3.head()

,base_date,time,nationality,city,emd_cd,emd_nm,visit_pop_cnt
0,2018-01-01,1,CHN,제주시,50110250,한림읍,66.3170
1,2018-01-01,1,ETC,제주시,50110250,한림읍,21.1308
2,2018-01-01,1,HKG,제주시,50110250,한림읍,3.6771
3,2018-01-01,1,JPN,제주시,50110250,한림읍,5.5962
4,2018-01-01,1,USA,제주시,50110250,한림읍,3.9078


In [5]:
# 필요없는 열 삭제 (날짜, 행정동코드, 행정동명, 배출량만 남김)
data2_3_pre = data2_3.drop(['time', 'nationality', 'city', 'emd_nm'], 1)
data2_3_pre

,base_date,emd_cd,visit_pop_cnt
0,2018-01-01,50110250,66.3170
1,2018-01-01,50110250,21.1308
2,2018-01-01,50110250,3.6771
3,2018-01-01,50110250,5.5962
4,2018-01-01,50110250,3.9078
...,...,...,...
6059039,2021-06-30,50130610,9.2120
6059040,2021-06-30,50130610,0.6039
6059041,2021-06-30,50130610,11.5473
6059042,2021-06-30,50130620,0.2013


In [6]:
# 각 열을 하나의 type으로 통일

# 날짜데이터 datetime으로 변환
data2_3_pre['base_date'] = pd.to_datetime(data2_3_pre['base_date'], format="%Y-%m-%d")

data2_3_pre.set_index(data2_3_pre['base_date'], inplace=True)

# "알수없음" 삭제
#idx = data2_3_pre[data2_3_pre['emd_cd'] == "XXXXXXXX"].index
#data2_3_pre = data2_3_pre.drop(idx)
# 행정동코드 int형으로 변환
data2_3_pre['emd_cd'] = pd.to_numeric(data2_3_pre['emd_cd'])
# 방문인구 int형으로 변환
data2_3_pre['visit_pop_cnt'] = pd.to_numeric(data2_3_pre['visit_pop_cnt'])
data2_3_pre

,base_date,emd_cd,visit_pop_cnt
base_date,,,
2018-01-01,2018-01-01,50110250,66.3170
2018-01-01,2018-01-01,50110250,21.1308
2018-01-01,2018-01-01,50110250,3.6771
2018-01-01,2018-01-01,50110250,5.5962
2018-01-01,2018-01-01,50110250,3.9078
...,...,...,...
2021-06-30,2021-06-30,50130610,9.2120
2021-06-30,2021-06-30,50130610,0.6039
2021-06-30,2021-06-30,50130610,11.5473


In [8]:
# 행정동코드 매개변수로 받아서 해당 구역의 배출량을 월별로 정리
def monthly_output(emdcd):
    emd_data = data2_3_pre[data2_3_pre['emd_cd'] == emdcd]    # 해당 행정동 데이터만 추출
    emd_data_monthly = emd_data.resample('M', on='base_date').sum()    # 월별로 합산
    emd_data_monthly['emd_cd'] = emdcd
    return emd_data_monthly

In [9]:
emd_cd_ary = [50110250, 50110253, 50110256, 50110259, 50110310, 50110320, 50110330, 50110510, 50110520, 50110530, 50110540, 50110550, 50110560, 50110570, 50110580, 50110590, 50110600, 50110610, 50110620, 50110630, 50110640, 50110650, 50110660, 50110670, 50110680, 50110690, 50130250, 50130253, 50130259, 50130310, 50130320, 50130510, 50130520, 50130530, 50130540, 50130550, 50130560, 50130570, 50130580, 50130590, 50130600, 50130610, 50130620]

data2_3_result = pd.DataFrame(index=range(0,0), columns=['base_date', 'emd_cd', 'visit_pop_cnt'])

for i in emd_cd_ary:
    tmp = monthly_output(i)
    data2_3_result = data2_3_result.append(tmp)

data2_3_result['base_date'] = data2_3_result.index  
data2_3_result

,base_date,emd_cd,visit_pop_cnt
2018-01-31,2018-01-31,50110250,58090.5960
2018-02-28,2018-02-28,50110250,69710.8394
2018-03-31,2018-03-31,50110250,98026.6381
2018-04-30,2018-04-30,50110250,103289.7891
2018-05-31,2018-05-31,50110250,115255.9750
...,...,...,...
2021-02-28,2021-02-28,50130620,280.6159
2021-03-31,2021-03-31,50130620,1475.8404
2021-04-30,2021-04-30,50130620,1283.0997
2021-05-31,2021-05-31,50130620,1212.2652


## 3 거주 인구 (월별 평균)

In [28]:
data3 = pd.read_csv("files\\data\\3_resident_pop.csv", encoding='cp949')
data3.head()

,base_year,base_month,city,emd_cd,emd_nm,sex,resid_reg_pop,foreign_pop,total_pop
0,2018,1,제주시,50110590,건입동,남성,5085,146,5231
1,2018,1,제주시,50110590,건입동,여성,4715,82,4797
2,2018,1,제주시,50110256,구좌읍,남성,7965,368,8333
3,2018,1,제주시,50110256,구좌읍,여성,7609,184,7793
4,2018,1,서귀포시,50130253,남원읍,남성,9806,428,10234


In [29]:
# 필요없는 열 삭제 (날짜, 행정동코드, 행정동명, 배출량만 남김)
data3_pre = data3.drop(['city', 'emd_nm', 'sex', 'resid_reg_pop', 'foreign_pop'], 1)
data3_pre

,base_year,base_month,emd_cd,total_pop
0,2018,1,50110590,5231
1,2018,1,50110590,4797
2,2018,1,50110256,8333
3,2018,1,50110256,7793
4,2018,1,50130253,10234
...,...,...,...,...
3607,2021,6,50110250,11481
3608,2021,6,50110600,12231
3609,2021,6,50110600,12399
3610,2021,6,50130550,2699


In [30]:
# 년, 월 병합
#df.insert(1,"base_date",['Lahore','Dehli','New York'],True)

data3_pre['base_year']= data3_pre['base_year'].astype('str')
data3_pre['base_month']= data3_pre['base_month'].astype('str')
data3_pre['total_pop']= data3_pre['total_pop'].astype('str')
data3_pre

,base_year,base_month,emd_cd,total_pop
0,2018,1,50110590,5231
1,2018,1,50110590,4797
2,2018,1,50110256,8333
3,2018,1,50110256,7793
4,2018,1,50130253,10234
...,...,...,...,...
3607,2021,6,50110250,11481
3608,2021,6,50110600,12231
3609,2021,6,50110600,12399
3610,2021,6,50130550,2699


In [31]:
data3_pre['base_date'] = data3_pre['base_year'] + "-" + data3_pre['base_month']
data3_pre

,base_year,base_month,emd_cd,total_pop,base_date
0,2018,1,50110590,5231,2018-1
1,2018,1,50110590,4797,2018-1
2,2018,1,50110256,8333,2018-1
3,2018,1,50110256,7793,2018-1
4,2018,1,50130253,10234,2018-1
...,...,...,...,...,...
3607,2021,6,50110250,11481,2021-6
3608,2021,6,50110600,12231,2021-6
3609,2021,6,50110600,12399,2021-6
3610,2021,6,50130550,2699,2021-6


In [32]:
data3_pre = data3_pre.drop(['base_year', 'base_month'], 1)
data3_pre2 = data3_pre[['base_date', 'emd_cd', 'total_pop']]
data3_pre2

,base_date,emd_cd,total_pop
0,2018-1,50110590,5231
1,2018-1,50110590,4797
2,2018-1,50110256,8333
3,2018-1,50110256,7793
4,2018-1,50130253,10234
...,...,...,...
3607,2021-6,50110250,11481
3608,2021-6,50110600,12231
3609,2021-6,50110600,12399
3610,2021-6,50130550,2699


In [34]:
# 각 열을 하나의 type으로 통일

# 날짜데이터 datetime으로 변환
data3_pre2['base_date'] = pd.to_datetime(data3_pre2['base_date'], format="%Y-%m-%d")

data3_pre2.set_index(data3_pre2['base_date'], inplace=True)

# "알수없음" 삭제
#idx = data3_pre2[data3_pre2['emd_cd'] == "XXXXXXXX"].index
#data3_pre2 = data3_pre2.drop(idx)
# 행정동코드 int형으로 변환
data3_pre2['emd_cd'] = pd.to_numeric(data3_pre2['emd_cd'])
# 총거주인구 int형으로 변환
data3_pre2['total_pop'] = pd.to_numeric(data3_pre2['total_pop'])
data3_pre2

,base_date,emd_cd,total_pop
base_date,,,
2018-01-01,2018-01-01,50110590,5231
2018-01-01,2018-01-01,50110590,4797
2018-01-01,2018-01-01,50110256,8333
2018-01-01,2018-01-01,50110256,7793
2018-01-01,2018-01-01,50130253,10234
...,...,...,...
2021-06-01,2021-06-01,50110250,11481
2021-06-01,2021-06-01,50110600,12231
2021-06-01,2021-06-01,50110600,12399


In [35]:
# 행정동코드 매개변수로 받아서 해당 구역의 배출량을 월별로 정리
def monthly_output2(emdcd):
    emd_data = data3_pre2[data3_pre2['emd_cd'] == emdcd]    # 해당 행정동 데이터만 추출
    emd_data_monthly = emd_data.resample('M', on='base_date').sum()    # 월별로 합산
    emd_data_monthly['emd_cd'] = emdcd
    return emd_data_monthly

In [36]:
emd_cd_ary = [50110250, 50110253, 50110256, 50110259, 50110310, 50110320, 50110330, 50110510, 50110520, 50110530, 50110540, 50110550, 50110560, 50110570, 50110580, 50110590, 50110600, 50110610, 50110620, 50110630, 50110640, 50110650, 50110660, 50110670, 50110680, 50110690, 50130250, 50130253, 50130259, 50130310, 50130320, 50130510, 50130520, 50130530, 50130540, 50130550, 50130560, 50130570, 50130580, 50130590, 50130600, 50130610, 50130620]

#data2_result = pd.DataFrame(index=range(0,0), columns=[])
data3_result = pd.DataFrame(index=range(0,0), columns=['base_date', 'emd_cd', 'total_pop'])

for i in emd_cd_ary:
    tmp = monthly_output2(i)
    data3_result = data3_result.append(tmp)

data3_result['base_date'] = data3_result.index  
data3_result

,base_date,emd_cd,total_pop
2018-01-31,2018-01-31,50110250,24419
2018-02-28,2018-02-28,50110250,24427
2018-03-31,2018-03-31,50110250,24532
2018-04-30,2018-04-30,50110250,24606
2018-05-31,2018-05-31,50110250,24715
...,...,...,...
2021-02-28,2021-02-28,50130620,4173
2021-03-31,2021-03-31,50130620,4178
2021-04-30,2021-04-30,50130620,4161
2021-05-31,2021-05-31,50130620,4178


In [37]:
merge_inner = pd.merge(data2_3_result, data3_result)
merge_inner

,base_date,emd_cd,visit_pop_cnt,total_pop
0,2018-01-31,50110250,58090.5960,24419
1,2018-02-28,50110250,69710.8394,24427
2,2018-03-31,50110250,98026.6381,24532
3,2018-04-30,50110250,103289.7891,24606
4,2018-05-31,50110250,115255.9750,24715
...,...,...,...,...
1801,2021-02-28,50130620,280.6159,4173
1802,2021-03-31,50130620,1475.8404,4178
1803,2021-04-30,50130620,1283.0997,4161
1804,2021-05-31,50130620,1212.2652,4178


In [40]:
merge_inner.to_csv("files\\data_preprocessing\\2_3_short_term_frgn_3_resident_pop_preprocessing.csv", index_label=False)

In [41]:
df = pd.read_csv("files\\data_preprocessing\\2_3_short_term_frgn_3_resident_pop_preprocessing.csv")
df

,base_date,emd_cd,visit_pop_cnt,total_pop
0,2018-01-31,50110250,58090.5960,24419
1,2018-02-28,50110250,69710.8394,24427
2,2018-03-31,50110250,98026.6381,24532
3,2018-04-30,50110250,103289.7891,24606
4,2018-05-31,50110250,115255.9750,24715
...,...,...,...,...
1801,2021-02-28,50130620,280.6159,4173
1802,2021-03-31,50130620,1475.8404,4178
1803,2021-04-30,50130620,1283.0997,4161
1804,2021-05-31,50130620,1212.2652,4178
